In [1]:
import torch
import scipy.special
from kan import KAN as BaseKAN,create_dataset
from better_kan import KAN as BetterKAN
import inspect
import numpy as np
from decimal import Decimal 
def convert_func(f):
    return lambda x:f(*x.unsqueeze(-1).permute(1,0,2))

def I_6_2(theta, sigma):
    return torch.exp(-theta**2 / (2 * sigma**2)) / torch.sqrt(2 * torch.pi * sigma**2)


def I_6_2b(theta, theta1, sigma):
    return torch.exp(-(theta - theta1)**2 / (2 * sigma**2)) / torch.sqrt(2 * torch.pi * sigma**2)


def I_9_18(a, b, c, d, e, f):
    return a / ((b - 1)**2 + (c - d)**2 + (e - f)**2)


def I_12_11(a, theta):
    return 1+a*torch.sin(theta)


def I_13_12(a,b):
    return a*(1/b-1)


def I_15_3x(a,b):
    # print(a.shape,b.shape)
    return (1-a)/torch.sqrt(1-b**2)

def I_16_6(a,b):
    return (a+b)/(1+a*b)

def I_18_4(a,b):
    return (1+a*b)/(1+a)

def I_26_2(n,theta2):
    return torch.arcsin(n*torch.sin(theta2))

def I_27_6(a,b):
    return 1/(1+a*b)

def I_29_16(a,theta1,theta2):
    return torch.sqrt(1+a**2-2*a*torch.cos(theta1-theta2))


def I_30_3(n,theta):
    return torch.sin(torch.sin(n*theta/2))/torch.sin(torch.sin(theta/2))

def I_30_5(a,n):
    return torch.arcsin(a/n)

def I_37_4(a,b):
    return 1+a*2*torch.sqrt(a)*torch.cos(b)
def I_40_1(n,a):
    return n*torch.exp(-a)
def I_44_4(n,b):
    return n*torch.log(b)
def I_50_26(a,alpha):
    return torch.cos(a)+alpha*torch.cos(a)**2
def II_2_42(a,b):
    return (a-1)*b
def II_6_15a(a,b,c):
    return 1/(4*torch.pi)*c*torch.sqrt(a**2+b**2)
def II_11_7(n,a,theta):
    return n*(1+a*torch.cos(theta))
def II_11_27(n,a):
    return (n*a)/(1-(n*a)/3)
def II_36_38(a,alpha,b):
    return a+alpha*b
def II_38_3(a,b):
    return a/b
def III_9_52(a,b,c):
    return a*torch.sin((b-c)/2)**2/torch.sin((b-c)/2)**2
def III_10_19(a,b):
    return torch.sqrt(1+a**2+b**2)
def III_17_37(a,b,theta):
    return b*(1+a*torch.cos(theta))
def ellipj(x,y):
    return scipy.special.ellipj(x,y)[0].real.float()

def ellipkinc(x,y):
    return scipy.special.ellipkinc(x,y)

def ellipeinc(x,y):
    return scipy.special.ellipeinc(x,y)

def jv(v,x):
    return scipy.special.jv(v,x)


def yv(v,x):
    return scipy.special.yv(v,x)


def iv(v,x):
    return scipy.special.iv(v,x)

def kv(v,x):
    return scipy.special.kv(v,x)

def lpmv0(m,x):
    return scipy.special.lpmv(0,m,x)

def lpmv1(m,x):
    return scipy.special.lpmv(1,m,x)

def lpmv2(m,x):
    return scipy.special.lpmv(2,m,x)

def sph_harm11(theta,phi):
    return scipy.special.sph_harm(1,1,theta,phi).real.float()

def sph_harm12(theta,phi):
    return scipy.special.sph_harm(1,2,theta,phi).real.float()

def sph_harm22(theta,phi):
    return scipy.special.sph_harm(2,2,theta,phi).real.float()

def sph_harm21(theta,phi):
    return scipy.special.sph_harm(2,1,theta,phi).real.float()
def test_accuracy_1(x):
    return torch.special.bessel_j0(x*20)
def test_accuracy_2(x,y):
    return torch.exp(torch.sin(torch.pi*x)+y**2)
def test_accuracy_3(x,y):
    return x*y
def test_accuracy_4(x1,x2,x3,x4):
    return torch.exp(torch.sin(x1**2+x2**2)+torch.cos(x3**2+x4**2))


def create_grid(start, end, spacing, dim):
    """
    Create a grid with specified start, end, spacing, and dimension.

    Parameters:
    start (float or list of floats): The starting value(s) for the grid.
    end (float or list of floats): The ending value(s) for the grid.
    spacing (float or list of floats): The spacing between points in the grid.
    dim (int): The dimension of the grid.

    Returns:
    torch.Tensor: The generated grid.
    """
    if isinstance(start, (int, float)):
        start = [start] * dim
    if isinstance(end, (int, float)):
        end = [end] * dim
    if isinstance(spacing, (int, float)):
        spacing = [spacing] * dim

    # Generate ranges for each dimension
    ranges = [torch.arange(start[i], end[i], spacing[i]) for i in range(dim)]

    # Create a meshgrid from the ranges
    grids = torch.meshgrid(*ranges, indexing='ij')

    # Stack the grids to form a grid of points
    grid = torch.stack(grids, dim=-1)
  
    return grid.reshape(-1, dim)
def create_n_grid(start, end, n, dim):
    assert dim>0
    if isinstance(start, (int, float)):
        start = [start] * dim
    if isinstance(end, (int, float)):
        end = [end] * dim
    n_each_dim= math.ceil(n**(1/dim))
    ranges = [torch.linspace(start[i], end[i], n_each_dim) for i in range(dim)]
    grids=torch.meshgrid(*ranges, indexing='ij')
    grid = torch.stack(grids, dim=-1)
    return grid.reshape(-1, dim)[:n]
class MLP(torch.nn.Module):
    def __init__(self, in_features, out_features, hidden_dim=128, num_layers=3, activation=torch.nn.SiLU(),seed=0):
        torch.manual_seed(seed)
        random.seed(seed)
        numpy.random.seed(seed)
        super(MLP, self).__init__()
        self.layers = nn.Sequential()
        self.layers.append(nn.Linear(in_features, hidden_dim))
        self.layers.append(activation)
        for _ in range(num_layers - 2):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim))
            self.layers.append(activation)
        self.layers.append(nn.Linear(hidden_dim, out_features))
    def forward(self, x):
        return self.layers(x)
    
def size_of_model(model):
    size=0
    for p in model.parameters():
        size+=p.numel()*p.element_size()
    return size
def params_of_model(model):
    return size_of_model(model)/8


In [2]:
from matplotlib import pyplot as plt
from kan.LBFGS import LBFGS
import itertools
from kan import  KAN,create_dataset
def test_kan(f,ranges,width,device="cuda",steps=50,seed=0,degrees=(5,10,20,50)):
    n_var=width[0]
    dataset=create_dataset(convert_func( f),n_var,ranges,device=device,train_num=3000)
    y=dataset["train_label"]
    print(y.min(),y.max()  )
    base_kan_test_losses=[]
    better_kan_test_losses=[]
    base_kan_train_losses=[]
    better_kan_train_losses=[]
    
 
        
    for deg in degrees:
        
        print(f"BetterKAN {deg}")
        kan=BetterKAN(width,deg,device=device,bias_trainable=False,base_fn=...).to(device)
        result=kan.train(dataset,lr=1,grid_update_num=10,steps=steps)
        better_kan_test_losses.extend(result["test_loss"])
        better_kan_train_losses.extend(result["train_loss"])
        # kan.plot()
        # # exit()
        
        print(f"BetterKAN {deg} 0.1")
        kan=BetterKAN(width,deg,device=device,seed=0,bias_trainable=False,base_fn=...).to(device)
        result=kan.train(dataset,lr=1/8,grid_update_num=10,steps=steps)
        
        better_kan_test_losses.extend(result["test_loss"])
        better_kan_train_losses.extend(result["train_loss"])
       
        kan=BaseKAN(width,deg,device=device).to(device)
        print(f"KAN {deg}")
        result=kan.train(dataset,device=device,steps=steps)
        base_kan_test_losses.extend(result["test_loss"])
        base_kan_train_losses.extend(result["train_loss"])
    # print(np.array(better_kan_test_losses),"array")
    base_kan_test_losses=np.nan_to_num( np.array(base_kan_test_losses),nan=float("inf"))
    better_kan_test_losses=np.nan_to_num( np.array(better_kan_test_losses),nan=float("inf"))
    base_kan_train_losses=np.nan_to_num( np.array(base_kan_train_losses),nan=float("inf"))
    better_kan_train_losses=np.nan_to_num( np.array(better_kan_train_losses),nan=float("inf"))
    # print(base_kan_test_losses)
    print(width)
    print("base kan test loss",'%.2E' % Decimal(base_kan_test_losses.min().astype(float)))
    print("better kan test loss",'%.2E' % Decimal(better_kan_test_losses.min().astype(float)))
    print("base kan train loss",'%.2E' % Decimal(base_kan_train_losses.min().astype(float)))
    print("better kan train loss",'%.2E' % Decimal(better_kan_train_losses.min().astype(float)))
    print("base kan test loss",'%.2E' % Decimal(base_kan_test_losses.min().astype(float)**2))
    print("better kan test loss",'%.2E' % Decimal(better_kan_test_losses.min().astype(float)**2))
    print("base kan train loss",'%.2E' % Decimal(base_kan_train_losses.min().astype(float)**2))
    print("better kan train loss",'%.2E' % Decimal(better_kan_train_losses.min().astype(float)**2))
    
    
        
       
def test(func,widths,input_min,input_max,steps=50,device="cuda",seed=0,name="",degrees=(5,10,20,50)):
    try:
        ranges=list(zip(input_min,input_max))
        test_kan(func,ranges,widths,device=device,steps=steps,seed=seed,degrees=degrees)
    except Exception as e:
        print(e)
        print("Failed")
        return
    
    

In [ ]:
test(ellipj,[2,2,1],[-1,0],[1,1])
test(ellipj, [2,3,2,1,1,1],[-1,0],[1,1])


In [ ]:
test(ellipkinc,[2,2,1,1],[-1,0],[1,1])
test(ellipkinc,[2,2,1,1,1],[-1,0],[1,1])

In [ ]:
test(ellipeinc,[2,2,1],[-1,0],[1,1])
test(ellipeinc,[2,2,1,1],[-1,0],[1,1])

In [4]:
test(jv,[2,2,1],[0,0],[1,1])
test(jv,[2,3,1,1,1],[0,0],[1,1])

tensor(0.0002, device='cuda:0') tensor(0.9965, device='cuda:0')
BetterKAN 5


train loss: 2.41e-03 | test loss: 2.42e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:29<00:00,  1.68it/s]


BetterKAN 5 0.1


train loss: 2.97e-03 | test loss: 2.91e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:29<00:00,  1.71it/s]


KAN 5


train loss: 4.00e-03 | test loss: 3.83e-03 | reg: 6.89e+00 : 100%|██| 50/50 [00:23<00:00,  2.15it/s]


BetterKAN 10


train loss: 2.02e-03 | test loss: 1.90e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:30<00:00,  1.63it/s]


BetterKAN 10 0.1


train loss: 3.64e-03 | test loss: 3.54e-03 | reg: 0.00e+00 :  46%|▉ | 23/50 [00:13<00:17,  1.56it/s]

In [17]:
test(yv,[2,2,2,1],[0,0],[0.1,1])
test(yv,[2,2,1],[0,0],[0.1,1])

tensor(-5.9793, device='cuda:0') tensor(0.0720, device='cuda:0')
BetterKAN 5


train loss: 9.67e-03 | test loss: 1.76e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:38<00:00,  1.31it/s]


BetterKAN 5 0.1


train loss: 8.76e-03 | test loss: 1.62e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:36<00:00,  1.36it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 3.62e-03 | test loss: 1.04e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:31<00:00,  1.59it/s]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [00:28<00:00,  1.76it/s]

base kan test loss 1.56E-02
better kan test loss 1.01E-02
base kan train loss 7.11E-03
better kan train loss 3.52E-03
base kan test loss 2.42E-04
better kan test loss 1.01E-04
base kan train loss 5.05E-05
better kan train loss 1.24E-05


In [18]:
test(kv,[2,2,1],[0,0],[0.1,1])
test(kv,[2,1,1],[0,0],[0.1,1])

tensor(0.4215, device='cuda:0') tensor(9.3096, device='cuda:0')
BetterKAN 5


train loss: 6.62e-03 | test loss: 1.93e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:34<00:00,  1.45it/s]


BetterKAN 5 0.1


train loss: 3.55e-02 | test loss: 4.28e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:30<00:00,  1.64it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 9.57e-03 | test loss: 2.08e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:31<00:00,  1.60it/s]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [00:18<00:00,  2.75it/s]

base kan test loss 2.72E-02
better kan test loss 1.74E-02
base kan train loss 1.49E-02
better kan train loss 9.48E-03
base kan test loss 7.40E-04
better kan test loss 3.04E-04
base kan train loss 2.21E-04
better kan train loss 8.98E-05


In [ ]:
test(iv,[2,4,3,2,1,1],[0,0],[1,1])


In [ ]:
test(lpmv0,[2,2,1],[-1,-1],[1,1])


In [19]:
test(lpmv1,[2,3,1],[0,0],[1,1])

tensor(-0.9997, device='cuda:0') tensor(-2.3353e-05, device='cuda:0')
BetterKAN 5


description:   0%|                                                           | 0/50 [00:00<?, ?it/s]

train loss: 1.25e-03 | test loss: 1.08e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:30<00:00,  1.64it/s]


BetterKAN 5 0.1


train loss: 2.24e-03 | test loss: 2.09e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:29<00:00,  1.67it/s]


...

BetterKAN 50 0.1


train loss: 1.64e-03 | test loss: 1.41e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:29<00:00,  1.70it/s]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [00:24<00:00,  2.06it/s]

base kan test loss 3.70E-04
better kan test loss 6.98E-04
base kan train loss 3.72E-04
better kan train loss 8.12E-04
base kan test loss 1.37E-07
better kan test loss 4.88E-07
base kan train loss 1.39E-07
better kan train loss 6.59E-07


In [20]:
test(lpmv2,[2,3,2,1],[0,0],[1,1])
test(lpmv2,[2,2,1],[0,0],[1,1])

tensor(-0.4130, device='cuda:0') tensor(-6.8425e-06, device='cuda:0')
BetterKAN 5


train loss: 1.37e-03 | test loss: 1.29e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:38<00:00,  1.32it/s]


BetterKAN 5 0.1


train loss: 3.08e-03 | test loss: 3.10e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:35<00:00,  1.42it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 4.48e-04 | test loss: 4.51e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:49<00:00,  1.00it/s]


KAN 50


train loss: 9.20e-04 | test loss: 1.09e-03 | reg: 6.39e+00 : 100%|██| 50/50 [00:45<00:00,  1.10it/s]

base kan test loss 7.95E-04
better kan test loss 3.77E-04
base kan train loss 8.01E-04
better kan train loss 3.78E-04
base kan test loss 6.31E-07
better kan test loss 1.42E-07
base kan train loss 6.42E-07
better kan train loss 1.43E-07


In [ ]:
# test(lpmv3,[2,3,2,1],[0,0],[1,1])
# test(lpmv3,[2,2,1],[0,0],[1,1])

In [ ]:
test(sph_harm11,[2,2,1],[0,0],[torch.pi,2*torch.pi])
test(sph_harm11,[2,3,2,1],[0,0],[torch.pi,2*torch.pi])

In [ ]:
test(sph_harm12,[2,2,1],[0,0],[torch.pi,2*torch.pi])
test(sph_harm12,[2,2,1,1],[0,0],[torch.pi,2*torch.pi])

In [ ]:
test(sph_harm22,[2,2,1],[0,0],[torch.pi,2*torch.pi])
test(sph_harm22,[2,2,3,2,1],[0,0],[torch.pi,2*torch.pi])

In [ ]:
test(lpmv0,[2,2,1],[0,0],[1,1])
test(lpmv0,[2,1,1],[0,0],[1,1])

In [ ]:
test(test_accuracy_1,[1,1],input_min= [-1],input_max= [1],steps=200,degrees=[5,10,20,50,100,200],seed=0,name="J_0(20x)")

In [ ]:
test(test_accuracy_2,[2,2,1],input_min= [-2,-1],input_max= [1,1],steps=200,degrees=[5,10,20,50,100,200],seed=0,name="exp(sin(pi*x)+y^2)")   

In [ ]:
test(test_accuracy_3,[2,3,1],input_min= [-1,-1],input_max= [1,1],steps=200,degrees=[5,10,20,50,100,200],seed=0,name="xy")

In [ ]:
test(test_accuracy_4,[4,2,1,1],input_min= [-1,-1],input_max= [1,1],steps=200,degrees=[5,10,20,50,100,200],seed=0,name="exp(sin(x1^2+x2^2)+cos(x3^2+x4^2)")    

In [ ]:
# test(I_6_2,[2,3,2,1,1,1],input_min=[-10,-10],input_max=[10,10])
test(I_6_2,[2,2,1,1],input_min=[-1,0.1],input_max=[1,1])
test(I_6_2,[2,3,2,1],input_min=[-1,0.1],input_max=[1,1])
test(I_6_2,[2,3,3,2,1],input_min=[-1,0.1],input_max=[1,1])

In [ ]:
# test(I_6_2b,[3,1,2,3,2,1],input_min=[-10,-10,-10],input_max=[10,10,10])
test(I_6_2b,[3,3,3,2,1],input_min=[-1,-1,0.1],input_max=[1,1,1])

test(I_6_2b,[3,3,2,1],input_min=[-1,-1,0.1],input_max=[1,1,1])

test(I_6_2b,[3,2,2,1,1],input_min=[-1,-1,0.1],input_max=[1,1,1])


In [ ]:
# test(I_9_18,[6,4,2,2,1],input_min=[-10,-10,-10,-10,-10,-10],input_max=[10,10,10,10,10,10])
test(I_9_18,[6,4,1,1],input_min=[-10,0,0.9,-1,0.9,-1],input_max=[10,0.9,1,-0.9,1,-0.9])

test(I_9_18,[6,4,2,1,1],input_min=[-10,0,0.9,-1,0.9,-1],input_max=[10,0.9,1,-0.9,1,-0.9])
# test(I_9_18,[6,4,1,1],input_min=[-10,-10,-10,-10,-10,-10],input_max=[10,10,10,10,10,10])

In [ ]:
test(I_12_11,[2,2,1],input_min=[-1,-torch.pi*2],input_max=[1,torch.pi*2])

test(I_12_11,[2,2,2,1],input_min=[-1,-torch.pi*2],input_max=[1,torch.pi*2])




In [ ]:
test(I_13_12,[2,2,1],input_min=[-1,0.5],input_max=[1,1])


In [ ]:
test(I_15_3x,[2,2,1],input_min=[-1,0],input_max=[1,0.99])
test(I_15_3x,[2,2,1,1,1],input_min=[-1,0],input_max=[1,0.99])

In [ ]:
test(I_16_6,[2,2,1],input_min=[0,0],input_max=[1,1])

test(I_16_6,[2,2,2,2,2,1],input_min=[0,0],input_max=[1,1])

In [21]:
test(I_18_4,[2,2,1],input_min=[0,0],input_max=[10,10])
test(I_18_4,[2,2,2,2,1],input_min=[0,0],input_max=[10,10])
test(I_18_4,[2,2,1],input_min=[0,0],input_max=[10,10])
test(I_18_4,[2,2,2,2,1],input_min=[0,0],input_max=[10,10])

tensor(0.0946, device='cuda:0') tensor(9.1280, device='cuda:0')
BetterKAN 5


train loss: 7.84e-03 | test loss: 8.02e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:53<00:00,  1.06s/it]


BetterKAN 5 0.1


train loss: 8.08e-03 | test loss: 7.85e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:52<00:00,  1.06s/it]


KAN 5


...

BetterKAN 50 0.1


train loss: 1.12e-02 | test loss: 1.05e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:46<00:00,  1.07it/s]


KAN 50


train loss: 1.62e-02 | test loss: 3.90e-02 | reg: 1.37e+01 : 100%|██| 50/50 [00:54<00:00,  1.10s/it]

base kan test loss 1.62E-02
better kan test loss 7.20E-03
base kan train loss 1.62E-02
better kan train loss 7.15E-03
base kan test loss 2.62E-04
better kan test loss 5.18E-05
base kan train loss 2.64E-04
better kan train loss 5.11E-05


In [22]:
test(I_26_2,[2,2,2,1,1],input_min=[-1,-torch.pi],input_max=[1,torch.pi])
test(I_26_2,[2,2,1],input_min=[-1,-2*torch.pi],input_max=[1,torch.pi])



tensor(-1.3747, device='cuda:0') tensor(1.4451, device='cuda:0')
BetterKAN 5


train loss: 6.12e-03 | test loss: 7.34e-03 | reg: 0.00e+00 : 100%|██| 50/50 [01:00<00:00,  1.22s/it]


BetterKAN 5 0.1


train loss: 8.77e-03 | test loss: 1.03e-02 | reg: 0.00e+00 : 100%|██| 50/50 [01:01<00:00,  1.23s/it]


KAN 5


...

BetterKAN 50 0.1


train loss: 1.55e-02 | test loss: 5.45e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:47<00:00,  1.06it/s]


KAN 50


train loss: 9.82e-02 | test loss: 1.38e-01 | reg: 5.20e+00 : 100%|██| 50/50 [00:35<00:00,  1.40it/s]

base kan test loss 6.21E-03
better kan test loss 4.44E-03
base kan train loss 6.14E-03
better kan train loss 2.73E-03
base kan test loss 3.86E-05
better kan test loss 1.97E-05
base kan train loss 3.76E-05
better kan train loss 7.48E-06


In [23]:
test(I_27_6,[2,2,1],input_min=[0,0],input_max=[10,10])
test(I_27_6,[2,2,2,1],input_min=[0,0],input_max=[10,10])
# test(I_27_6,[2,2,1],input_min=[0,0],input_max=[1,1])
# test(I_27_6,[2,2,2,1],input_min=[0,0],input_max=[1,1])

tensor(0.0103, device='cuda:0') tensor(0.9999, device='cuda:0')
BetterKAN 5


train loss: 2.34e-03 | test loss: 2.23e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:50<00:00,  1.02s/it]


BetterKAN 5 0.1


train loss: 1.43e-03 | test loss: 1.26e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:51<00:00,  1.02s/it]


KAN 5


...

BetterKAN 50 0.1


train loss: 9.92e-04 | test loss: 8.85e-04 | reg: 0.00e+00 : 100%|██| 50/50 [01:08<00:00,  1.38s/it]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [01:23<00:00,  1.67s/it]

base kan test loss 2.44E-03
better kan test loss 8.85E-04
base kan train loss 1.77E-03
better kan train loss 7.63E-04
base kan test loss 5.95E-06
better kan test loss 7.83E-07
base kan train loss 3.13E-06
better kan train loss 5.81E-07


In [24]:
# test(I_30_3,[2,4,3,1],input_min=[-1,-2*torch.pi],input_max=[10,2*torch.pi])
# test(I_30_3,[2,2,3,2,1,1],input_min=[-1,-2*torch.pi],input_max=[10,2*torch.pi])
test(I_30_3,[2,4,3,1],input_min=[-torch.pi,-torch.pi+0.001],input_max=[torch.pi,torch.pi-0.001])
test(I_30_3,[2,2,3,2,1,1],input_min=[-10,-torch.pi+0.001],input_max=[10,torch.pi-0.001])
test(I_30_3,[2,4,3,1],input_min=[-1,-torch.pi+0.001],input_max=[1,torch.pi-0.001])
test(I_30_3,[2,2,3,2,1,1],input_min=[-1,-torch.pi+0.001],input_max=[1,torch.pi-0.001])

tensor(-3.0673, device='cuda:0') tensor(3.1031, device='cuda:0')
BetterKAN 5


train loss: 8.33e-03 | test loss: 8.95e-03 | reg: 0.00e+00 : 100%|██| 50/50 [01:33<00:00,  1.87s/it]


BetterKAN 5 0.1


train loss: 1.47e-02 | test loss: 1.55e-02 | reg: 0.00e+00 : 100%|██| 50/50 [01:30<00:00,  1.82s/it]


KAN 5


...

BetterKAN 50 0.1


train loss: 5.84e-04 | test loss: 5.98e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:52<00:00,  1.05s/it]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [01:12<00:00,  1.45s/it]

base kan test loss 1.28E-03
better kan test loss 3.39E-04
base kan train loss 1.18E-03
better kan train loss 3.30E-04
base kan test loss 1.63E-06
better kan test loss 1.15E-07
base kan train loss 1.39E-06
better kan train loss 1.09E-07


In [25]:
test(sph_harm11,[2,1,1],input_min=[0,0],input_max=[torch.pi,2*torch.pi])
test(sph_harm11,[2,3,2,1],input_min=[0,0],input_max=[torch.pi,2*torch.pi])

tensor(-0.3454, device='cuda:0') tensor(0.3447, device='cuda:0')
BetterKAN 5


train loss: 5.04e-02 | test loss: 5.09e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:18<00:00,  2.73it/s]


BetterKAN 5 0.1


train loss: 5.94e-02 | test loss: 6.01e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:25<00:00,  1.94it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 1.12e-03 | test loss: 1.14e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:34<00:00,  1.45it/s]


KAN 50


train loss: 1.11e-03 | test loss: 1.46e-03 | reg: 1.21e+01 : 100%|██| 50/50 [00:35<00:00,  1.39it/s]

base kan test loss 1.22E-03
better kan test loss 6.45E-04
base kan train loss 1.10E-03
better kan train loss 6.35E-04
base kan test loss 1.49E-06
better kan test loss 4.16E-07
base kan train loss 1.21E-06
better kan train loss 4.03E-07


In [26]:
test(I_30_5, [2,1,1],input_min=[-1,1],input_max=[1,10])
test(I_30_5,  [2,1,1,1,1,1],input_min=[-1,1],input_max=[1,10])

tensor(-1.3649, device='cuda:0') tensor(1.3425, device='cuda:0')
BetterKAN 5


train loss: 5.89e-02 | test loss: 6.70e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:24<00:00,  2.01it/s]


BetterKAN 5 0.1


train loss: 8.44e-02 | test loss: 8.73e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:19<00:00,  2.57it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 1.02e-01 | test loss: 1.24e-01 | reg: 0.00e+00 : 100%|██| 50/50 [00:31<00:00,  1.58it/s]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [00:43<00:00,  1.14it/s]

base kan test loss 7.02E-02
better kan test loss 6.38E-02
base kan train loss 6.17E-02
better kan train loss 5.68E-02
base kan test loss 4.93E-03
better kan test loss 4.07E-03
base kan train loss 3.81E-03
better kan train loss 3.23E-03


In [45]:
test(I_37_4, [2,3,2,1],input_min=[0,-torch.pi],input_max=[1,torch.pi])
test(I_37_4,  [2,2,1],input_min=[0,-torch.pi],input_max=[1,torch.pi])


tensor(-0.9967, device='cuda:0') tensor(2.9824, device='cuda:0')
BetterKAN 5


train loss: 7.26e-04 | test loss: 7.27e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:44<00:00,  1.12it/s]


BetterKAN 5 0.1


train loss: 1.06e-03 | test loss: 1.06e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:42<00:00,  1.17it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 3.64e-04 | test loss: 3.66e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:31<00:00,  1.61it/s]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [00:25<00:00,  1.99it/s]

base kan test loss 3.37E-03
better kan test loss 2.21E-04
base kan train loss 3.27E-03
better kan train loss 1.99E-04
base kan test loss 1.14E-05
better kan test loss 4.90E-08
base kan train loss 1.07E-05
better kan train loss 3.94E-08


In [46]:
test(I_40_1, [2,2,1],input_min=[-1,0],input_max=[1,1])
test(I_40_1,  [2,1,1],input_min=[-1,0],input_max=[1,1])

tensor(-0.9965, device='cuda:0') tensor(0.9939, device='cuda:0')
BetterKAN 5


train loss: 9.92e-05 | test loss: 9.65e-05 | reg: 0.00e+00 : 100%|██| 50/50 [00:31<00:00,  1.59it/s]


BetterKAN 5 0.1


train loss: 2.18e-04 | test loss: 2.17e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:30<00:00,  1.67it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 4.71e-02 | test loss: 5.23e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:27<00:00,  1.83it/s]


KAN 50


train loss: 5.15e-02 | test loss: 6.11e-02 | reg: 2.36e+00 : 100%|██| 50/50 [00:23<00:00,  2.16it/s]

base kan test loss 5.86E-02
better kan test loss 4.99E-02
base kan train loss 5.09E-02
better kan train loss 4.71E-02
base kan test loss 3.43E-03
better kan test loss 2.49E-03
base kan train loss 2.59E-03
better kan train loss 2.22E-03


In [47]:
test(I_44_4, [2,2,1],input_min=[-1,0],input_max=[1,1])

tensor(-7.9293, device='cuda:0') tensor(6.0879, device='cuda:0')
BetterKAN 5


train loss: 3.15e-02 | test loss: 2.98e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:24<00:00,  2.03it/s]


BetterKAN 5 0.1


train loss: 4.69e-02 | test loss: 4.74e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:30<00:00,  1.66it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 1.84e-02 | test loss: 2.09e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:31<00:00,  1.57it/s]


KAN 50


train loss: 1.13e-01 | test loss: 3.41e-01 | reg: 6.94e+00 : 100%|██| 50/50 [00:25<00:00,  1.99it/s]

base kan test loss 3.29E-02
better kan test loss 1.32E-02
base kan train loss 3.55E-02
better kan train loss 1.32E-02
base kan test loss 1.08E-03
better kan test loss 1.74E-04
base kan train loss 1.26E-03
better kan train loss 1.74E-04


In [ ]:
def train_acc():
    return torch.mean(((model(dataset['train_input']) > 0.5) == dataset['train_label']).float())

def test_acc():
    return torch.mean(((model(dataset['test_input']) > 0.5) == dataset['test_label']).float())
# print(dataset['test_input'].shape)
model = KAN(width=[n_feature,1,1],num_basis=5)
model.fix_symbolic(1,0,0,'gaussian',fit_params_bool=False)
model.train(dataset, lamb=0.001, batch=1024, metrics=[train_acc, test_acc],lr=1);
print(test_acc(), train_acc())

train loss: 1.20e-01 | test loss: 1.58e-01 | reg: 1.35e+01 : 100%|█| 100/100 [01:54<00:00,  1.15s/it


tensor(0.9718) tensor(0.9720)


In [48]:
test(I_50_26,  [2,2,3,1],input_min=[-torch.pi,0],input_max=[torch.pi,1])
test(I_50_26, [2,3,1],input_min=[-torch.pi,0],input_max=[torch.pi,1])


tensor(-0.9977, device='cuda:0') tensor(1.9753, device='cuda:0')
BetterKAN 5


train loss: 1.55e-03 | test loss: 1.63e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:42<00:00,  1.19it/s]


BetterKAN 5 0.1


train loss: 5.71e-03 | test loss: 5.71e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:39<00:00,  1.26it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 2.28e-04 | test loss: 2.25e-04 | reg: 0.00e+00 : 100%|██| 50/50 [01:14<00:00,  1.49s/it]


KAN 50


train loss: 6.42e-03 | test loss: 8.99e-03 | reg: 8.51e+00 : 100%|██| 50/50 [01:19<00:00,  1.59s/it]

base kan test loss 2.18E-03
better kan test loss 6.03E-05
base kan train loss 1.95E-03
better kan train loss 5.99E-05
base kan test loss 4.76E-06
better kan test loss 3.64E-09
base kan train loss 3.80E-06
better kan train loss 3.58E-09


In [49]:
test(II_2_42, [2,2,3,1],input_min=[-1,0],input_max=[1,1])
test(II_2_42,  [2,3,1],input_min=[-1,0],input_max=[1,1])  

tensor(-1.9383, device='cuda:0') tensor(-6.9501e-06, device='cuda:0')
BetterKAN 5


train loss: 2.83e-03 | test loss: 2.72e-03 | reg: 0.00e+00 : 100%|██| 50/50 [01:33<00:00,  1.86s/it]


BetterKAN 5 0.1


train loss: 1.13e-03 | test loss: 1.19e-03 | reg: 0.00e+00 : 100%|██| 50/50 [01:28<00:00,  1.77s/it]


KAN 5


...

BetterKAN 50 0.1


train loss: 1.31e-04 | test loss: 1.44e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:47<00:00,  1.05it/s]


KAN 50


train loss: 5.70e-04 | test loss: 1.82e-03 | reg: 9.99e+00 : 100%|██| 50/50 [00:36<00:00,  1.39it/s]

base kan test loss 8.29E-04
better kan test loss 7.19E-05
base kan train loss 5.70E-04
better kan train loss 6.86E-05
base kan test loss 6.87E-07
better kan test loss 5.17E-09
base kan train loss 3.25E-07
better kan train loss 4.71E-09


In [50]:
test(II_6_15a, [3,2,2,2,1],input_min=[-1,-1,-1],input_max=[1,1,1])
test(II_6_15a, [3,2,1,1],input_min=[-1,-1,-1],input_max=[1,1,1])

tensor(-0.1066, device='cuda:0') tensor(0.1019, device='cuda:0')
BetterKAN 5


description:   0%|                                                           | 0/50 [00:00<?, ?it/s]

train loss: 6.54e-04 | test loss: 8.34e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:51<00:00,  1.03s/it]


BetterKAN 5 0.1


train loss: 1.10e-03 | test loss: 1.32e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:40<00:00,  1.24it/s]


...

BetterKAN 50 0.1


train loss: 3.46e-04 | test loss: 5.01e-04 | reg: 0.00e+00 : 100%|██| 50/50 [01:00<00:00,  1.21s/it]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [01:15<00:00,  1.52s/it]

base kan test loss 6.07E-03
better kan test loss 3.59E-04
base kan train loss 5.78E-03
better kan train loss 2.52E-04
base kan test loss 3.68E-05
better kan test loss 1.29E-07
base kan train loss 3.34E-05
better kan train loss 6.33E-08


In [51]:
test(II_11_7, [3,3,3,2,2,1],input_min=[-1,-1,-torch.pi],input_max=[1,1,torch.pi])
test(II_11_7, [3,3,1,1],input_min=[-1,-1,-torch.pi],input_max=[1,1,torch.pi])

tensor(-1.8916, device='cuda:0') tensor(1.8337, device='cuda:0')
BetterKAN 5


train loss: 1.31e-02 | test loss: 1.33e-02 | reg: 0.00e+00 : 100%|██| 50/50 [01:09<00:00,  1.39s/it]


BetterKAN 5 0.1


train loss: 4.60e-03 | test loss: 4.73e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:52<00:00,  1.05s/it]


KAN 5


...

BetterKAN 50 0.1


train loss: 4.41e-03 | test loss: 4.88e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:38<00:00,  1.29it/s]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [00:38<00:00,  1.30it/s]

base kan test loss 2.02E-02
better kan test loss 4.88E-03
base kan train loss 1.89E-02
better kan train loss 4.41E-03
base kan test loss 4.09E-04
better kan test loss 2.38E-05
base kan train loss 3.57E-04
better kan train loss 1.94E-05


In [52]:
test(II_11_27,  [2,2,1,2,1],input_min=[-1,0],input_max=[1,1])
test(II_11_27,[2,1,1],input_min=[-1,0],input_max=[1,1])

tensor(-0.7309, device='cuda:0') tensor(1.3950, device='cuda:0')
BetterKAN 5


train loss: 6.09e-04 | test loss: 5.94e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:39<00:00,  1.26it/s]


BetterKAN 5 0.1


train loss: 2.55e-03 | test loss: 2.44e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:23<00:00,  2.09it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 3.09e-02 | test loss: 5.27e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:23<00:00,  2.14it/s]


KAN 50


train loss: 2.56e-02 | test loss: 4.40e-02 | reg: 2.55e+00 : 100%|██| 50/50 [00:17<00:00,  2.80it/s]

base kan test loss 4.37E-02
better kan test loss 2.34E-02
base kan train loss 2.56E-02
better kan train loss 2.46E-02
base kan test loss 1.91E-03
better kan test loss 5.47E-04
base kan train loss 6.54E-04
better kan train loss 6.03E-04


In [53]:
test(II_36_38,  [3,3,1],input_min=[-1,-1,-1],input_max=[1,1,1])
test(II_36_38,  [3,2,1],input_min=[-1,-1,-1],input_max=[1,1,1])

tensor(-1.8229, device='cuda:0') tensor(1.7454, device='cuda:0')
BetterKAN 5


train loss: 1.32e-03 | test loss: 1.40e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:30<00:00,  1.63it/s]


BetterKAN 5 0.1


train loss: 2.34e-03 | test loss: 2.50e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:29<00:00,  1.69it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 3.96e-04 | test loss: 4.21e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:55<00:00,  1.11s/it]


KAN 50


train loss: 1.26e-01 | test loss: 1.75e-01 | reg: 6.74e+00 : 100%|██| 50/50 [00:52<00:00,  1.05s/it]

base kan test loss 1.57E-03
better kan test loss 2.19E-04
base kan train loss 4.08E-04
better kan train loss 1.94E-04
base kan test loss 2.46E-06
better kan test loss 4.78E-08
base kan train loss 1.66E-07
better kan train loss 3.74E-08


In [7]:
test(II_38_3,  [2,2,1],input_min=[-1,0.1],input_max=[1,1])
test(II_38_3,  [2,2,1,1,1],input_min=[-1,0.1],input_max=[1,1])

tensor(-9.7887, device='cuda:0') tensor(9.7406, device='cuda:0')
BetterKAN 5


train loss: 3.37e-03 | test loss: 3.53e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:26<00:00,  1.87it/s]


BetterKAN 5 0.1


train loss: 7.63e-03 | test loss: 7.60e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:27<00:00,  1.84it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 3.81e-02 | test loss: 3.21e-01 | reg: 0.00e+00 : 100%|██| 50/50 [00:32<00:00,  1.53it/s]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [00:42<00:00,  1.17it/s]

[2, 2, 1, 1, 1]
base kan test loss 5.42E-02
better kan test loss 2.30E-02
base kan train loss 4.29E-02
better kan train loss 1.90E-02
base kan test loss 2.94E-03
better kan test loss 5.29E-04
base kan train loss 1.84E-03
better kan train loss 3.61E-04


In [4]:
test(III_9_52,  [3,2,3,1,1],input_min=[-1,-1,-1],input_max=[1,1,1])
test(III_9_52,  [3,3,2,1,1,1],input_min=[-1,-1,-1],input_max=[1,1,1])

tensor(-0.9998, device='cuda:0') tensor(0.9997, device='cuda:0')
BetterKAN 5


train loss: 9.47e-05 | test loss: 9.30e-05 | reg: 0.00e+00 : 100%|██| 50/50 [00:32<00:00,  1.53it/s]


BetterKAN 5 0.1


train loss: 9.76e-05 | test loss: 9.98e-05 | reg: 0.00e+00 : 100%|██| 50/50 [00:32<00:00,  1.53it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 1.82e-02 | test loss: 7.50e-02 | reg: 0.00e+00 : 100%|██| 50/50 [01:18<00:00,  1.58s/it]


KAN 50


train loss: nan | test loss: nan | reg: nan : 100%|█████████████████| 50/50 [01:40<00:00,  2.02s/it]

[3, 3, 2, 1, 1, 1]
base kan test loss 5.88E-04
better kan test loss 1.50E-04
base kan train loss 3.47E-04
better kan train loss 1.56E-04
base kan test loss 3.46E-07
better kan test loss 2.25E-08
base kan train loss 1.21E-07
better kan train loss 2.44E-08


In [5]:
test(III_10_19,  [2,1,1],input_min=[-1,0],input_max=[1,1])
test(III_10_19,  [2,1,2,1],input_min=[-1,0],input_max=[1,1])

tensor(1.0004, device='cuda:0') tensor(1.7127, device='cuda:0')
BetterKAN 5


train loss: 1.66e-04 | test loss: 1.65e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:19<00:00,  2.57it/s]


BetterKAN 5 0.1


train loss: 5.71e-04 | test loss: 5.81e-04 | reg: 0.00e+00 : 100%|██| 50/50 [00:22<00:00,  2.18it/s]


KAN 5


...

BetterKAN 50 0.1


train loss: 4.35e-02 | test loss: 4.41e-02 | reg: 0.00e+00 : 100%|██| 50/50 [00:48<00:00,  1.03it/s]


KAN 50


train loss: 3.26e-02 | test loss: 4.59e-02 | reg: 6.06e+00 : 100%|██| 50/50 [00:54<00:00,  1.08s/it]

[2, 1, 2, 1]
base kan test loss 1.04E-04
better kan test loss 5.34E-05
base kan train loss 1.02E-04
better kan train loss 5.53E-05
base kan test loss 1.09E-08
better kan test loss 2.85E-09
base kan train loss 1.04E-08
better kan train loss 3.06E-09


In [6]:
test( III_17_37,  [3,3,3,2,2,1],input_min=[-1,-1,-torch.pi],input_max=[1,1,torch.pi])
test( III_17_37,  [3,3,1],input_min=[-1,-1,-torch.pi],input_max=[1,1,torch.pi])

tensor(-1.9617, device='cuda:0') tensor(1.8881, device='cuda:0')
BetterKAN 5


description:   0%|                                                           | 0/50 [00:00<?, ?it/s]

train loss: 6.69e-03 | test loss: 6.79e-03 | reg: 0.00e+00 : 100%|██| 50/50 [01:38<00:00,  1.96s/it]


BetterKAN 5 0.1


train loss: 4.95e-03 | test loss: 5.21e-03 | reg: 0.00e+00 : 100%|██| 50/50 [01:32<00:00,  1.86s/it]


...

BetterKAN 50 0.1


train loss: 8.28e-03 | test loss: 8.44e-03 | reg: 0.00e+00 : 100%|██| 50/50 [00:29<00:00,  1.70it/s]


KAN 50


train loss: 8.90e-02 | test loss: 1.49e-01 | reg: 8.36e+00 : 100%|██| 50/50 [00:26<00:00,  1.88it/s]

[3, 3, 1]
base kan test loss 2.38E-02
better kan test loss 5.58E-03
base kan train loss 2.41E-02
better kan train loss 5.52E-03
base kan test loss 5.68E-04
better kan test loss 3.11E-05
base kan train loss 5.79E-04
better kan train loss 3.05E-05
